In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_58466/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 06:29:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/17 06:29:04 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data
data = {
    'employee_id': [1, 9, 2, 11, 13, 4],
    'experience': ['Junior', 'Junior', 'Senior', 'Senior', 'Senior', 'Junior'],
    'salary': [10000, 15000, 20000, 16000, 50000, 40000]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Candidates")

import pandas as pd

# Sample data
data = {
    'employee_id': [1, 9, 2, 11, 13, 4],
    'experience': ['Junior', 'Junior', 'Senior', 'Senior', 'Senior', 'Junior'],
    'salary': [25000, 10000, 85000, 80000, 90000, 30000]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Candidates_1")


   employee_id experience  salary
0            1     Junior   10000
1            9     Junior   15000
2            2     Senior   20000
3           11     Senior   16000
4           13     Senior   50000
5            4     Junior   40000
   employee_id experience  salary
0            1     Junior   25000
1            9     Junior   10000
2            2     Senior   85000
3           11     Senior   80000
4           13     Senior   90000
5            4     Junior   30000


In [6]:
query = """
    with senior_salary as (
        select
            *,
            sum(salary) over(order by rk asc) as total_salary
        from (
            select
                *,
                row_number() over(order by salary asc) as rk
            from Candidates c 
            where experience = 'Senior'
        )
    ),
    
    junior_salary as (
        select
            *,
            sum(salary) over(order by rk asc) as total_salary
        from (
            select
                *,
                row_number() over(order by salary asc) as rk
            from Candidates c 
            where experience = 'Junior'
        )
    ),
    
    senior_get as (
        select
            *
        from senior_salary 
        where total_salary <= 70000
    )
    
    select 
        employee_id
    from junior_salary 
    where total_salary <= (70000 - ifnull((select max(total_salary) from senior_get), 0))
    union all
    select
        employee_id
    from senior_get
"""

In [7]:
spark.sql(query).show()

24/03/17 06:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:31:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 0

+-----------+
|employee_id|
+-----------+
|          1|
|          9|
|         11|
|          2|
+-----------+

